In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from pandas_profiling import ProfileReport

In [183]:
# Similar keys
# region
# province
# barangay
# precinct_id
# city/municipality(?)

# Contains the election results of the 2022 Philippine presidential elections at a per-precinct level.
pres = pd.read_csv("Elections Dataset/pres_precinct_data_2022.csv")
pres.rename(columns = {
    'Region':'region',
    'Province':'province',
    'Barangay':'barangay',
    "Precinct ID":'precinct_id',
    'City':'city'
}, inplace=True)

# Contains the election results of the 2016 Philippine vice-presidential elections at a per-precinct level. 
vp = pd.read_csv("Elections Dataset/vp_precinct_data_2016.csv")

# Contains precinct descriptions and polling centers centers for the 2022 Philippine presidential elections.
population = pd.read_excel("Elections Dataset/2022NLEPOP.xlsx")
population.rename(columns = {
    'REGION':'region',
    'PROVINCE':'province',
    'BARANGAY':'barangay',
    "PRECINCT_ID":'precinct_id',
    'MUNICIPALITY':'city'
}, inplace=True)


In [184]:
print(pres.shape)

(105666, 15)


In [185]:
pres.head()

,region,province,city,barangay,precinct_id,"ABELLA, ERNIE (IND)","DE GUZMAN, LEODY (PLM)","DOMAGOSO, ISKO MORENO (AKSYON)","GONZALES, NORBERTO (PDSP)","LACSON, PING (PDR)","MANGONDATO, FAISAL (KTPNAN)","MARCOS, BONGBONG (PFP)","MONTEMAYOR, JOSE JR. (DPP)","PACQUIAO, MANNY PACMAN(PROMDI)","ROBREDO, LENI (IND)"
0,REGION X,BUKIDNON,BAUNGON,BALINTAD,13010009,2,1,4,0,0,1,224,0,115,29
1,REGION X,BUKIDNON,BAUNGON,BUENAVISTA,13010010,3,5,7,5,0,2,310,1,188,47
2,REGION X,BUKIDNON,BAUNGON,DANATAG,13010011,4,3,9,4,3,0,288,4,152,44
3,REGION X,BUKIDNON,BAUNGON,DANATAG,13010012,2,2,16,2,3,0,190,3,160,51
4,REGION X,BUKIDNON,BAUNGON,DANATAG,13010013,0,1,16,4,6,0,272,2,125,46


In [186]:
population.shape

(107785, 9)

In [187]:
population.head()

,region,province,city,barangay,precinct_id,CLUSTER,CLUSTERTOTAL,CLUSTERED_PRECINCTS,POLLINGCENTER
0,BARMM,BASILAN,AKBAR,LINONGAN,7080001,1,423,"0012A, 0012P1, 0011A","LINONGAN ELEMENTARY SCHOOL, BARANGAY LINONGAN,..."
1,BARMM,BASILAN,AKBAR,LINONGAN,7080002,2,411,"0011B, 0011C, 0012B","LINONGAN ELEMENTARY SCHOOL, BARANGAY LINONGAN,..."
2,BARMM,BASILAN,AKBAR,MANGUSO,7080003,3,313,"0035A, 0035C, 0035P1, 0035B","AKBAR ELEMENTARY SCHOOL, BARANGAY UPPER BATO-B..."
3,BARMM,BASILAN,AKBAR,UPPER SINANGKAPAN,7080004,4,655,"0050A, 0050B, 0050C, 0050D, 0050P1",BARANGAY UPPER SINANGKAPAN COVERED COURT (BUSC...
4,BARMM,BASILAN,AKBAR,CADDAYAN,7080005,5,479,"0054A, 0055A, 0055B, 0055C, 0055P1","CADDAYAN ELEMENTARY SCHOOL, BARANGAY CADDAYAN,..."


In [122]:
# Precint w/o votes

president_prec_id = pres.loc[:,"Precinct ID"]
population_prec_id = population.loc[:,"PRECINCT_ID"]

# bool series
prec_id_match = population_prec_id.isin(president_prec_id)

prec_id_no_match = population_prec_id[~prec_id_match]

print(f"Precint IDs w/o match \n{no_match}")

# why no votes?
# may precints din ba na hindi nag vote nung 2016 na nag vote now? where is the data?

Precint IDs w/o match 
16         7090004
38         7010004
64         7010030
66         7010032
67         7010033
            ...   
106571    67240146
106580    67240155
106581    67240156
106589    67240164
106590    67240165
Name: PRECINCT_ID, Length: 2119, dtype: int64


In [177]:
# city and municipality equality check

president_prec_id = pres.loc[:,"City"]
population_prec_id = population.loc[:,"MUNICIPALITY"]

# bool series
prec_id_match = population_prec_id.isin(president_prec_id)

prec_id_no_match = population_prec_id[~prec_id_match]

print(f"Precint IDs w/o match \n{prec_id_no_match}")

Precint IDs w/o match 
767       BUTIG
768       BUTIG
769       BUTIG
770       BUTIG
771       BUTIG
          ...  
58351    SIBUCO
58352    SIBUCO
58353    SIBUCO
58354    SIBUCO
58355    SIBUCO
Name: MUNICIPALITY, Length: 436, dtype: object


In [142]:
no_match = population["PRECINCT_ID"].isin(prec_id_no_match)

# inverse no match to get matched
population_matched = population[~no_match]

# population_matched.to_csv("2022NLEPOP_matched.csv", index=False)


In [6]:
vp.head()

,region,province,municipality,barangay,clustered_precinct,cayetano,escudero,honasan,marcos,robredo,trillanes,registered_voters,ballots_cast,precincts,polling_center,timestamp
0,REGION I,PANGASINAN,CALASIAO,BUENLAG,55170026,25,47,8,230,138,8,592,503,"0060A, 0060B, 0061A, 0062A",BUENLAG ELEMENTARY SCHOOL,05/09/2016 19:11:37
1,REGION I,PANGASINAN,CALASIAO,BUENLAG,55170027,28,52,3,191,137,7,526,458,"0062B, 0063A, 0063B, 0064A",BUENLAG ELEMENTARY SCHOOL,05/09/2016 18:39:48
2,REGION I,PANGASINAN,CALASIAO,BUENLAG,55170023,32,75,5,295,156,12,723,649,"0051C, 0052A, 0052B, 0053A, 0053B",BUENLAG ELEMENTARY SCHOOL,05/09/2016 18:36:49
3,REGION I,PANGASINAN,CALASIAO,BUENLAG,55170022,50,91,5,291,172,14,768,650,"0049A, 0050A, 0050B, 0051A, 0051B",BUENLAG ELEMENTARY SCHOOL,05/09/2016 17:12:03
4,REGION I,PANGASINAN,CALASIAO,BUENLAG,55170025,58,96,3,303,162,14,787,662,"0056C, 0057A, 0057B, 0058A, 0058B, 0059A, 0059B",BUENLAG ELEMENTARY SCHOOL,05/09/2016 18:34:08


In [6]:
mean = pres["ROBREDO, LENI (IND)"].mean()
trim_mean = stats.trim_mean(pres["ROBREDO, LENI (IND)"], 0.1)
median = pres["ROBREDO, LENI (IND)"].median()

print(f"Mean {mean}, Trimmed mean {trim_mean}, Median {median}")

Mean 139.92561467264778, Trimmed mean 125.82562046040647, Median 121.0


In [189]:
merged_2022 = pd.merge(pres, population, how="left", on=["region","province","barangay","precinct_id","city"])

# merged_2022.to_csv("merged_2022.csv", index=False)

In [164]:
pres_keys = list(map(lambda x : x.lower(), pres.keys()))
pop_keys = list(map(lambda x : x.lower(), population.keys()))

for x in pres_keys:
    if x in pop_keys:
        print(x)

region
province
barangay
precinct_id


In [222]:
merged_2022 = pd.read_csv("merged_2022.csv")

In [215]:
merged_2022.loc[merged_2022["precinct_id"] == 39101380]

,region,province,city,barangay,precinct_id,"ABELLA, ERNIE (IND)","DE GUZMAN, LEODY (PLM)","DOMAGOSO, ISKO MORENO (AKSYON)","GONZALES, NORBERTO (PDSP)","LACSON, PING (PDR)","MANGONDATO, FAISAL (KTPNAN)","MARCOS, BONGBONG (PFP)","MONTEMAYOR, JOSE JR. (DPP)","PACQUIAO, MANNY PACMAN(PROMDI)","ROBREDO, LENI (IND)",CLUSTER,CLUSTERTOTAL,CLUSTERED_PRECINCTS,POLLINGCENTER
13325,NATIONAL CAPITAL REGION,NCR - MANILA,MALATE,BARANGAY 725,39101380,0,0,73,0,7,0,112,0,1,251,NaN,NaN,NaN,NaN


In [192]:
pres["precinct_id"]

0         13010009
1         13010010
2         13010011
3         13010012
4         13010013
            ...   
105661     8120018
105662     8120049
105663     8120050
105664     8120003
105665     8120004
Name: precinct_id, Length: 105666, dtype: int64

In [212]:
# check kung match lahat
# merged_2022 precinct ids == pres_precinct_data ids

x = merged_2022["precinct_id"].tolist()
y = pres["precinct_id"].tolist()

print(x == y)
        

True


In [213]:
profile = ProfileReport(merged_2022, title="merged_2022", explorative=True)
profile.to_file("merged_2022.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\Martin\AppData\Local\pypoetry\Cache\virtualenvs\pjdsc-1LJUQi52-py3.10\lib\site-packages\scipy\stats\_stats_py.py:4868: RuntimeWarning: overflow encountered in longlong_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [223]:
merged_2022.head()

,region,province,city,barangay,precinct_id,"ABELLA, ERNIE (IND)","DE GUZMAN, LEODY (PLM)","DOMAGOSO, ISKO MORENO (AKSYON)","GONZALES, NORBERTO (PDSP)","LACSON, PING (PDR)","MANGONDATO, FAISAL (KTPNAN)","MARCOS, BONGBONG (PFP)","MONTEMAYOR, JOSE JR. (DPP)","PACQUIAO, MANNY PACMAN(PROMDI)","ROBREDO, LENI (IND)",CLUSTER,CLUSTERTOTAL,CLUSTERED_PRECINCTS,POLLINGCENTER
0,REGION X,BUKIDNON,BAUNGON,BALINTAD,13010009,2,1,4,0,0,1,224,0,115,29,9.0,486.0,"0014A, 0015A, 0015B, 0014B","BALINTAD, ELEMENTARY SCHOOL, BALINTAD, BAUNGON..."
1,REGION X,BUKIDNON,BAUNGON,BUENAVISTA,13010010,3,5,7,5,0,2,310,1,188,47,10.0,769.0,"0016A, 0016B, 0017A, 0017B, 0017C","BUENAVISTA, ELEMENTARY SCHOOL, BUENAVISTA, BAU..."
2,REGION X,BUKIDNON,BAUNGON,DANATAG,13010011,4,3,9,4,3,0,288,4,152,44,11.0,645.0,"0018A, 0018B, 0019A, 0019B","DANATAG, ELEMENTARY SCHOOL, DANATAG, BAUNGON, ..."
3,REGION X,BUKIDNON,BAUNGON,DANATAG,13010012,2,2,16,2,3,0,190,3,160,51,12.0,592.0,"0020A, 0020B, 0021A, 0021B, 0024A","DANATAG, ELEMENTARY SCHOOL, DANATAG, BAUNGON, ..."
4,REGION X,BUKIDNON,BAUNGON,DANATAG,13010013,0,1,16,4,6,0,272,2,125,46,13.0,593.0,"0022A, 0022B, 0023A, 0023B","DANATAG, ELEMENTARY SCHOOL, DANATAG, BAUNGON, ..."


In [227]:
merged_2022.shape

(105666, 19)

In [226]:
# remove null values from merged_2022
merged_2022_cleaned = merged_2022.dropna()


In [228]:
merged_2022_cleaned.shape

(90944, 19)

In [229]:
merged_2022_cleaned.head()

,region,province,city,barangay,precinct_id,"ABELLA, ERNIE (IND)","DE GUZMAN, LEODY (PLM)","DOMAGOSO, ISKO MORENO (AKSYON)","GONZALES, NORBERTO (PDSP)","LACSON, PING (PDR)","MANGONDATO, FAISAL (KTPNAN)","MARCOS, BONGBONG (PFP)","MONTEMAYOR, JOSE JR. (DPP)","PACQUIAO, MANNY PACMAN(PROMDI)","ROBREDO, LENI (IND)",CLUSTER,CLUSTERTOTAL,CLUSTERED_PRECINCTS,POLLINGCENTER
0,REGION X,BUKIDNON,BAUNGON,BALINTAD,13010009,2,1,4,0,0,1,224,0,115,29,9.0,486.0,"0014A, 0015A, 0015B, 0014B","BALINTAD, ELEMENTARY SCHOOL, BALINTAD, BAUNGON..."
1,REGION X,BUKIDNON,BAUNGON,BUENAVISTA,13010010,3,5,7,5,0,2,310,1,188,47,10.0,769.0,"0016A, 0016B, 0017A, 0017B, 0017C","BUENAVISTA, ELEMENTARY SCHOOL, BUENAVISTA, BAU..."
2,REGION X,BUKIDNON,BAUNGON,DANATAG,13010011,4,3,9,4,3,0,288,4,152,44,11.0,645.0,"0018A, 0018B, 0019A, 0019B","DANATAG, ELEMENTARY SCHOOL, DANATAG, BAUNGON, ..."
3,REGION X,BUKIDNON,BAUNGON,DANATAG,13010012,2,2,16,2,3,0,190,3,160,51,12.0,592.0,"0020A, 0020B, 0021A, 0021B, 0024A","DANATAG, ELEMENTARY SCHOOL, DANATAG, BAUNGON, ..."
4,REGION X,BUKIDNON,BAUNGON,DANATAG,13010013,0,1,16,4,6,0,272,2,125,46,13.0,593.0,"0022A, 0022B, 0023A, 0023B","DANATAG, ELEMENTARY SCHOOL, DANATAG, BAUNGON, ..."


In [230]:
merged_2022_cleaned.to_csv("merged_2022_cleaned.csv", index=False)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [232]:
merged_2022_cleaned = pd.read_csv("merged_2022_cleaned.csv")
profile = ProfileReport(merged_2022_cleaned, title="merged_2022_cleaned", explorative=True)
profile.to_file("merged_2022_cleaned.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]